In [226]:
import pandas as pd
import numpy as np

In [227]:
df = pd.read_csv("scrapper/UK_raw_data.csv")
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Science,£24K - £38K (Glassdoor est.),Love using technology to solve complex problem...,4.1,MThree Consulting\n4.1,"London, England",-1,501 to 1000 Employees,2010,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,-1
1,Data Science Graduate Scheme,£24K - £38K (Glassdoor est.),"Salary and Benefits: £24,500 + pension + 25 da...",3.6,Yorkshire Water\n3.6,"Bradford, Yorkshire and the Humber, England",-1,1001 to 5000 Employees,1973,Company - Private,Utilities,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),-1
2,Data Engineer,£24K - £38K (Glassdoor est.),Job Description\n\nData Engineer\n\nAt Citadel...,4.3,Citadel\n4.3,"London, England",-1,1001 to 5000 Employees,1990,Company - Private,-1,-1,$50 to $100 million (USD),-1
3,Data Scientist (Investigations),£24K - £38K (Glassdoor est.),Who are we?\n\nHi! �� We are Ravelin! We're a ...,4.8,Ravelin\n4.8,"London, England",-1,51 to 200 Employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,$5 to $10 million (USD),-1
4,Data/Marketing Analyst,£24K - £38K (Glassdoor est.),Data / Marketing Analyst\n\nThe Marketing team...,4.6,gohenry Ltd\n4.6,"London, England",-1,51 to 200 Employees,2012,Company - Private,Investment Banking & Asset Management,Finance,$10 to $25 million (USD),-1


In [228]:
df.shape
df.drop_duplicates(inplace=True)

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 880 entries, 0 to 932
Data columns (total 14 columns):
Job Title            880 non-null object
Salary Estimate      880 non-null object
Job Description      880 non-null object
Rating               880 non-null float64
Company Name         880 non-null object
Location             880 non-null object
Headquarters         880 non-null int64
Size                 880 non-null object
Founded              880 non-null int64
Type of ownership    880 non-null object
Industry             880 non-null object
Sector               880 non-null object
Revenue              880 non-null object
Competitors          880 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 103.1+ KB


#### 1. Salary Estimate Parsing

In [230]:
# df['hourly'] = df['Salary Estimate'].apply(lambda x : 1 if 'per hour' in x.lower() else 0)
# df['hourly'].value_counts() --> NO Hourly Salaries in this dateset
# df['employer provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)
# df['employer provided']. value_counts() --> NO such data

df = df[df['Salary Estimate'] != '-1'] 
salary = df['Salary Estimate'].apply(lambda x:x.split("(")[0])
remove_symbols = salary.apply(lambda x: x.replace('K','').replace('£',''))
print(salary[0])
print(remove_symbols[0])

£24K - £38K 
24 - 38 


In [231]:
df['min_salary'] = remove_symbols.apply(lambda x: int(x.split('-')[0]))
print(df['min_salary'][0])
df['max_salary'] = remove_symbols.apply(lambda x: int(x.split('-')[1]))
print(df['max_salary'][0])
df['avg_salary'] = (df['min_salary'] + df['max_salary']) / 2

24
38


#### 2. Company Name

In [232]:
df['Company Name'] = df.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'].split('\n')[0], axis=1)
print(df['Company Name'][0:2])

0    MThree Consulting
1      Yorkshire Water
Name: Company Name, dtype: object


In [233]:
df.drop(['Headquarters', 'Competitors'], axis=1, inplace=True)

In [234]:
df.head(1)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary
0,Data Science,£24K - £38K (Glassdoor est.),Love using technology to solve complex problem...,4.1,MThree Consulting,"London, England",501 to 1000 Employees,2010,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,24,38,31.0


#### 3. Job Description

In [235]:
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
print(df['python'].value_counts())

df['sql'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
print(df['sql'].value_counts())

df['r_studio'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
print(df['r_studio'].value_counts())

df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
print(df['spark'].value_counts())

df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
print(df['aws'].value_counts())

df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.excel.value_counts()

0    693
1    187
Name: python, dtype: int64
0    694
1    186
Name: sql, dtype: int64
0    879
1      1
Name: r_studio, dtype: int64
0    839
1     41
Name: spark, dtype: int64
0    790
1     90
Name: aws, dtype: int64


1    548
0    332
Name: excel, dtype: int64

In [236]:
# Job description length
df['description_len'] = df['Job Description'].apply(lambda x: len(x))
df['description_len'][0:5]

0    3098
1    3750
2    2899
3    3388
4    3376
Name: description_len, dtype: int64

#### 4. Founded --> age

In [237]:
df['age'] = df['Founded'].apply(lambda x: x if x<1 else 2021-x)
df['age'][0:5]

0    11
1    48
2    31
3     7
4     9
Name: age, dtype: int64

#### 5. Size --> employees

In [238]:
df['Size'].value_counts()

10000+ Employees           279
1001 to 5000 Employees     149
51 to 200 Employees        122
1 to 50 Employees           97
-1                          70
5001 to 10000 Employees     60
201 to 500 Employees        45
501 to 1000 Employees       35
Unknown                     23
Name: Size, dtype: int64

#### 6. Job Title

In [239]:
def title_simplifier(title):
    if 'data scientist' in title.lower(): return 'data scientist'
    elif 'data engineer' in title.lower(): return 'data engineer'
    elif 'analyst' in title.lower(): return 'analyst'
    elif 'machine learning' in title.lower(): return 'mle'
    elif 'manager' in title.lower(): return 'manager'
    elif 'director' in title.lower(): return 'director'
    elif 'consultant' in title.lower(): return 'consultant'
    elif 'researcher' in title.lower(): return 'ai researcher'
    else: return 'na'

def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'junior'
    else:
        return 'na'
    

In [240]:
df['job_title'] = df['Job Title'].apply(title_simplifier)
print(df['job_title'].value_counts())

df['seniority'] = df['Job Title'].apply(seniority)
print(df['seniority'].value_counts())

na                425
manager           172
director           88
analyst            82
data scientist     59
ai researcher      22
consultant         19
data engineer      12
mle                 1
Name: job_title, dtype: int64
na        676
senior    204
Name: seniority, dtype: int64


In [241]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,python,sql,r_studio,spark,aws,excel,description_len,age,job_title,seniority
0,Data Science,£24K - £38K (Glassdoor est.),Love using technology to solve complex problem...,4.1,MThree Consulting,"London, England",501 to 1000 Employees,2010,Company - Private,IT Services,...,1,0,0,0,0,0,3098,11,na,na
1,Data Science Graduate Scheme,£24K - £38K (Glassdoor est.),"Salary and Benefits: £24,500 + pension + 25 da...",3.6,Yorkshire Water,"Bradford, Yorkshire and the Humber, England",1001 to 5000 Employees,1973,Company - Private,Utilities,...,0,0,0,0,0,0,3750,48,na,na
2,Data Engineer,£24K - £38K (Glassdoor est.),Job Description\n\nData Engineer\n\nAt Citadel...,4.3,Citadel,"London, England",1001 to 5000 Employees,1990,Company - Private,-1,...,1,1,0,1,1,0,2899,31,data engineer,na
3,Data Scientist (Investigations),£24K - £38K (Glassdoor est.),Who are we?\n\nHi! �� We are Ravelin! We're a ...,4.8,Ravelin,"London, England",51 to 200 Employees,2014,Company - Private,Enterprise Software & Network Solutions,...,0,1,0,0,0,0,3388,7,data scientist,na
4,Data/Marketing Analyst,£24K - £38K (Glassdoor est.),Data / Marketing Analyst\n\nThe Marketing team...,4.6,gohenry Ltd,"London, England",51 to 200 Employees,2012,Company - Private,Investment Banking & Asset Management,...,0,1,0,0,0,1,3376,9,analyst,na


#### 7. Industry

In [242]:
df['Industry'].unique()

array(['IT Services', 'Utilities', '-1',
       'Enterprise Software & Network Solutions',
       'Investment Banking & Asset Management',
       'Biotech & Pharmaceuticals', 'Staffing & Outsourcing',
       'Food & Beverage Stores', 'Internet', 'Video Games',
       'Other Retail Stores', 'General Merchandise & Superstores',
       'Airlines', 'Insurance Carriers',
       'Financial Transaction Processing', 'Consulting',
       'Computer Hardware & Software', 'Federal Agencies', 'Energy',
       'Hotels, Motels, & Resorts', 'Logistics & Supply Chain',
       'Oil & Gas Exploration & Production',
       'Consumer Products Manufacturing', 'Banks & Credit Unions',
       'Research & Development', 'Advertising & Marketing',
       'Health Care Services & Hospitals', 'Telecommunications Services',
       'Architectural & Engineering Services', 'Drug & Health Stores',
       'Aerospace & Defense', 'Electrical & Electronic Manufacturing',
       'Lending', 'Industrial Manufacturing', 'Publis

In [243]:
def industry_simplifier(typ):
    typ = typ.lower()
    if 'IT' in typ or 'software' in typ or 'games' in typ: return 'inforamtion tech'
    elif 'banking' in typ or 'financial' in typ or 'insurance' in typ or 'lending' in typ or 'accounting' in typ or 'banks' in typ : return 'finance'
    elif 'pharmaceuticals' in typ or 'health' in typ: return 'medical'
    elif 'stores' in typ or 'superstores' in typ: return 'retail'
    elif 'production' in typ or 'manufacturing' in typ or 'manufacture' in typ: return 'production'
    elif 'education' in typ or 'universities' in typ: return 'education'
    elif 'hotels' in typ : return 'hospitality'
    elif 'cable' in typ or 'internet' in typ or 'advertising' in typ or 'telecommunications' in typ: return 'telecommunication'
    elif 'federal' in typ or 'state' in typ or 'securtiy' in typ or 'consulting' or 'social' in typ in typ: return 'public'
    else: return 'na'

In [244]:
df['sector'] = df['Industry'].apply(industry_simplifier)

In [245]:
df['sector'].value_counts()

public               329
medical              198
inforamtion tech     105
telecommunication     75
finance               62
education             41
production            35
retail                34
hospitality            1
Name: sector, dtype: int64

#### 8. Size

In [246]:
df['Size'].value_counts()

10000+ Employees           279
1001 to 5000 Employees     149
51 to 200 Employees        122
1 to 50 Employees           97
-1                          70
5001 to 10000 Employees     60
201 to 500 Employees        45
501 to 1000 Employees       35
Unknown                     23
Name: Size, dtype: int64

Filtering the 'Size' feature using fractional imputation

In [247]:
def FilterDataset(Feature):
    featuredata_list = df[Feature].unique().tolist()
    data_count = df[Feature].value_counts().sum()
    Null = df[Feature].isnull().sum()
    mask_mat = df[Feature].isna()
    Frac = 0
    for data in featuredata_list:
        if Null > 0:
            Frac = (len(df[df[Feature] == data])/data_count)
            data_count = data_count - len(df[df[Feature] == data])
            ind = df[Feature].loc[mask_mat].sample(frac=Frac, replace=True).index
            df.loc[ind, Feature] = data
            mask_mat = df[Feature].isna()
            Null = df[Feature].isnull().sum()
    else:print(Feature + " Sorted")

In [248]:
# replacing -1 and Unknowns with np.nan
df['Size'].replace({'-1' : np.nan, 'Unknown':np.nan}, inplace=True)

In [249]:
df['Size'].isnull().sum()

93

In [252]:
FilterDataset('Size')

Size Sorted


In [253]:
df['Size'].value_counts()

10000+ Employees           307
1001 to 5000 Employees     165
51 to 200 Employees        135
1 to 50 Employees          109
5001 to 10000 Employees     73
201 to 500 Employees        52
501 to 1000 Employees       39
Name: Size, dtype: int64

In [254]:
# sparsing the 'Size'
employees = df['Size'].apply(lambda x:x.split("+")[0] if '+' in x else x.split('E')[0])
print(employees.value_counts())
df['min_employees'] = employees.apply(lambda x: int(x.split('to')[0]))
print(df['min_employees'][0:5])
df['max_employees'] = employees.apply(lambda x: int(x.split('to')[1]) if 'to' in x else int(x))
print(df['max_employees'][0:5])

10000             307
1001 to 5000      165
51 to 200         135
1 to 50           109
5001 to 10000      73
201 to 500         52
501 to 1000        39
Name: Size, dtype: int64
0     501
1    1001
2    1001
3      51
4      51
Name: min_employees, dtype: int64
0    1000
1    5000
2    5000
3     200
4     200
Name: max_employees, dtype: int64


In [255]:
# Taking mean/average of min_employees and max_employess
df['avg_size'] = round((df['min_employees']+df['max_employees']) / 2 ,0)
print(df['avg_size'][0:2])

0     750.0
1    3000.0
Name: avg_size, dtype: float64


#### 9. Revenue

In [256]:
print(df['Revenue'].value_counts(),'\n___________________________________')
# mapping '-1' to np.nan
df['Revenue'].replace({'-1':np.nan}, inplace=True)
print(df['Revenue'].isnull().sum())
# Fractional Impuatation of NaN values
FilterDataset('Revenue')
print(df['Revenue'].isnull().sum())



Unknown / Non-Applicable            184
$10+ billion (USD)                  166
Less than $1 million (USD)           97
$100 to $500 million (USD)           84
-1                                   70
$2 to $5 billion (USD)               66
$500 million to $1 billion (USD)     50
$10 to $25 million (USD)             38
$5 to $10 billion (USD)              32
$25 to $50 million (USD)             30
$50 to $100 million (USD)            26
$1 to $2 billion (USD)               25
$1 to $5 million (USD)               10
$5 to $10 million (USD)               2
Name: Revenue, dtype: int64 
___________________________________
70
Revenue Sorted
2


#### 10. Type of ownership

In [257]:
df['Type of ownership'].value_counts()

Company - Private                 392
Company - Public                  311
-1                                 70
College / University               29
Subsidiary or Business Segment     28
Government                         16
Unknown                            14
Hospital                            9
Nonprofit Organization              6
Other Organization                  3
Contract                            1
Private Practice / Firm             1
Name: Type of ownership, dtype: int64

In [258]:
# mapping -1 to np.nan and others to relevant/common names
df['Type of ownership'].replace({'Company - Private':'Private', 'Private Practice / Firm':'Private',
                                 'Company - Public': 'Public', '-1':np.nan,
                                 'Subsidiary or Business Segment':'Venture', 'Contract':'Venture',
                                 'Unknown': 'Organization', 'Other Organization':'Organization', 'Nonprofit Organization':'Organization'}, inplace=True)
print(df['Type of ownership'].value_counts(),'\n____________________')
print(df['Type of ownership'].isnull().sum())

Private                 393
Public                  311
Venture                  29
College / University     29
Organization             23
Government               16
Hospital                  9
Name: Type of ownership, dtype: int64 
____________________
70


In [261]:
# Fractional imputation
FilterDataset('Type of ownership')

Type of ownership Sorted


In [262]:
print(df['Type of ownership'].isnull().sum(),'\n___________________________')
print(df['Type of ownership'].value_counts())

0 
___________________________
Private                 424
Public                  331
College / University     33
Venture                  32
Organization             28
Government               20
Hospital                 12
Name: Type of ownership, dtype: int64


In [263]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'min_salary', 'max_salary',
       'avg_salary', 'python', 'sql', 'r_studio', 'spark', 'aws', 'excel',
       'description_len', 'age', 'job_title', 'seniority', 'sector',
       'min_employees', 'max_employees', 'avg_size'],
      dtype='object')

In [264]:
final_df = df

In [266]:
final_df.drop(['Job Title','Salary Estimate', 'Job Description','Size', 'Founded', 'Location','Industry'], axis=1, inplace=True)
final_df.columns

Index(['Rating', 'Company Name', 'Type of ownership', 'Sector', 'Revenue',
       'min_salary', 'max_salary', 'avg_salary', 'python', 'sql', 'r_studio',
       'spark', 'aws', 'excel', 'description_len', 'age', 'job_title',
       'seniority', 'sector', 'min_employees', 'max_employees', 'avg_size'],
      dtype='object')